Prompts, chaining, and prompt engineering is important. However, you might not always know what chain or prompts you need to execute prior to receiving user input or new data. This is where automation and **agents** can help. This is an active area of development, but some very useful tooling is available.

In the following we will explore using [LangChain agents](https://python.langchain.com/en/latest/use_cases/personal_assistants.html) and [Prediction Guard's Task Agents]() to detect and automate LLM actions.

# Dependencies and imports

In [1]:
! pip install predictionguard langchain google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.6 MB/s eta 0:00:00
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32002 sha256=5a568b00af2f40637e244e02b0f4579a10c63df43874a7bcccd45619bf28a549
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully

In [4]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import PredictionGuard
import os
from getpass import getpass

In [5]:
pg_access_token = getpass('Enter your Prediction Guard access token: ')
os.environ['PREDICTIONGUARD_TOKEN'] = pg_access_token

Enter your Prediction Guard access token: ··········


In [6]:
serpapi_key = getpass('Enter your serpapi api key: ')
os.environ['SERPAPI_API_KEY'] = serpapi_key

Enter your serpapi api key: ··········


In [13]:
openai_api_key = getpass('Enter your OpenAI api key: ')
os.environ['OPENAI_API_KEY'] = openai_api_key

Enter your OpenAI api key: ··········


# Searching the Internet on-the-fly

In this example, we will use the SerpAPI to perform Google searches on-the-fly based on an LLM parsing of user input. You will need a SerpAPI key to run this example. You can setup a free account and get a key [here](https://serpapi.com/).

In [14]:
# In LangChain, "tools" are like resources that are available to your agent to
# execute certain actions (like a Google Search) while trying to complete a
# set of tasks. An "agent" is the object that you "run" to try and get a "Final Answer."
tools = load_tools(["serpapi"], llm=PredictionGuard(model="OpenAI-text-davinci-003"))
agent = initialize_agent(tools, PredictionGuard(model="OpenAI-text-davinci-003"),
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [15]:
agent.run("How are Domino's gift cards delivered?")



> Entering new  chain...
 I need to find out how Domino's gift cards are delivered.
Action: Search
Action Input: "Domino's gift cards delivery"
Observation: Order gift cards & eGift cards from Domino's in electronic or plastic. ... Each comes with a personalized card carrier and is delivered via US Mail.
Thought: So it looks like Domino's gift cards are delivered via US Mail.
Final Answer: Domino's gift cards are delivered via US Mail.

> Finished chain.


"Domino's gift cards are delivered via US Mail."

In [ ]:
agent.run("Find the Domino's pizza Guarantee. Translate that into German adding a sassy comment about the guarantee.")



> Entering new AgentExecutor chain...
 I need to research Domino's Pizza Guarantee and then translate it
Action: Search
Action Input: Domino's Pizza Guarantee
Observation: Our Guarantee: If you are not completely satisfied with your Domino's Pizza experience, we will make it right or refund your money. Domino's Carryout Insurance ...
Thought: I need to translate this to German
Action: Search
Action Input: Domino's Pizza Guarantee translation German
Observation: Our Guarantee: If you are not completely satisfied with your Domino's Pizza experience, we will make it right or refund your money. Domino's Carryout Insurance ...
Thought: I now know the final answer
Final Answer: Unsere Garantie: Wenn Sie mit Ihrer Domino's Pizza Erfahrung nicht vollständig zufrieden sind, machen wir es richtig oder erstatten Ihnen Ihr Geld zurück. Domino's Carryout Insurance ... Mit der Garantie von Domino's Pizza kannst du sicher sein, dass du immer eine leckere Pizza bekommst - oder dein Geld zurück!

> F

"Unsere Garantie: Wenn Sie mit Ihrer Domino's Pizza Erfahrung nicht vollständig zufrieden sind, machen wir es richtig oder erstatten Ihnen Ihr Geld zurück. Domino's Carryout Insurance ... Mit der Garantie von Domino's Pizza kannst du sicher sein, dass du immer eine leckere Pizza bekommst - oder dein Geld zurück!"